# USDOT and GEH Calculation - routeSampler

In [24]:
import os
import sys
import pathlib
from pathlib import Path

raw_path = pathlib.Path().absolute()  # a bit of Jupyter Magic to get the working path
PATH = str(raw_path.parent.parent)
if PATH not in sys.path:
    print(f"adding {PATH} to path")
    sys.path.append(PATH)

In [25]:
import json5 as json
import pickle
# import swifter
from datetime import timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np


In [26]:
def read_json(path):
    with open(path, 'r') as f:
        return json.load(f)

def day_filter(df_like, date_low, date_high, time_series=None):
    time_obj = time_series if time_series is not None else df_like.index        
    indexer = (time_obj >= pd.to_datetime(date_low)) & (time_obj < pd.to_datetime(date_high))
    return df_like.loc[indexer]

def time_filter(df_like, date_low, date_high, time_series=None):
    try:
        time_obj = time_series.dt.time if time_series is not None else df_like.index
    except AttributeError:
        time_obj = time_series.time

    time_low = pd.to_datetime(date_low).time()
    time_high = pd.to_datetime(date_high).time()
    indexer = (time_obj >= time_low) & (time_obj <= time_high)
    time_low_dt = pd.to_datetime(date_low)
    return df_like.loc[indexer], pd.Series(time_series).apply(lambda x: x.replace(day=time_low_dt.day, month=time_low_dt.month, year=time_low_dt.year)).loc[indexer]

In [27]:
# need to use the input file method for this
experiment_path = Path(
    "/home/max/tmp/sumo-uc-2023/CarFollowingDefaults/02.06.2023_17.39.54"
)
DETECTOR_OUTPUT_CSV = experiment_path / '55' / 'detectors.parquet'


TRAFFIC_LIGHTS = ['63082002', '63082003', '63082004']
AIRPORT, LOWES, HARPER = TRAFFIC_LIGHTS

START_TIME = pd.to_datetime('2020-02-24 05:30:00')
END_TIME = pd.to_datetime('2020-02-24 11:50:00') #.replace(hour=23, minute=59, second=59)

AGG_INTERVAL = 600

## Get the RW Data

In [28]:
rw_df = pd.read_parquet("02_24_2020_cleaned.parquet")

### Convert the SUMO Data to the same format as the RW Data


In [29]:
sumo_df = pd.read_parquet(
    DETECTOR_OUTPUT_CSV,
)
# append _interval to the columns
sumo_df.columns = [f"interval_{col}" for col in sumo_df.columns]

In [30]:
# create the timestamp column
sumo_df["Timestamp"] = sumo_df["interval_begin"].apply(
    lambda x: START_TIME + timedelta(seconds=x)
)
# make the columns the interval_id & the values the count
sumo_df = (
    sumo_df[["Timestamp", "interval_id", "interval_nVehEntered"]]
    .set_index(["Timestamp", "interval_id"])
    .unstack(
        [
            "interval_id",
        ]
    )
)
# drop the multiindex
sumo_df.columns = sumo_df.columns.droplevel(0)
# reset the index
sumo_df = sumo_df.reset_index()

# sum together detectors that are the "same", (ie. 1_1 and 1_2)
columns = sumo_df.columns.difference(["Timestamp"])
two_level = [
    ("_".join(split_col[:-1]), col)
    for col, split_col in zip(columns, columns.str.split("_"))
]

for short_col, long_col in two_level:
    if short_col in sumo_df.columns:
        sumo_df[short_col] = sumo_df[short_col] + sumo_df[long_col]
    else:
        sumo_df[short_col] = sumo_df[long_col]
    sumo_df.drop(long_col, axis=1, inplace=True)

sumo_df.set_index("Timestamp", inplace=True)


### Resample the RW data to match the sumo data


In [31]:
# resample the rw_df to the sumo_df interval
rw_df = rw_df.resample('1S').sum()
sql_df_day = rw_df.index.mean()
sql_df_day


Timestamp('2020-02-24 11:59:50.499999744', freq='S')

In [32]:
rw_df  = rw_df.resample(f'{AGG_INTERVAL}S').sum()
sumo_df = sumo_df.resample(f'{AGG_INTERVAL}S').sum()

rw_df = rw_df.reindex_like(
    sumo_df
)


START_TIME = rw_df.index.min()
END_TIME = rw_df.index.max()

### Create the EB and WB Columns


In [33]:
for tl in ["63082002", "63082003", "63082004"]:
    for ext, detectors in [("EB", [9, 10]), ("WB", [11, 12])]:
        sumo_df[f"{tl}_{ext}"] = sumo_df[[f"{tl}_{d}" for d in detectors]].sum(axis=1)
        rw_df[f"{tl}_{ext}"] = rw_df[[f"{tl}_{d}" for d in detectors]].sum(axis=1)


In [34]:
keep_cols = sumo_df.columns.intersection(rw_df.columns)
sumo_df = sumo_df[keep_cols]
rw_df = rw_df[keep_cols]

## Load the Historical Analysis File

In [35]:
HISTORICAL_AGG_INTERVAL = 300

with open(f'detector_average_table_{HISTORICAL_AGG_INTERVAL}S.pkl', 'rb') as f:
    average_table = pickle.load(f)

# dealing with stupid datetimes
average_table['time_index'] = pd.to_datetime([time.strftime('%H:%M:%S') for time in average_table['time_index']])

In [36]:
# filter to our time range
for tl, tl_dict in average_table.items():
    if tl in TRAFFIC_LIGHTS:
        print(tl)
        for detect, detect_dict in tl_dict.items():
            average_table[tl][detect]['counts'], time_index = time_filter(detect_dict['counts'].copy(), START_TIME, END_TIME, average_table['time_index'].copy())
            # resample the historic data to match AGG_INTERVAL
            average_table[tl][detect]['counts'] = average_table[tl][detect]['counts'].set_index(time_index).resample(f"{AGG_INTERVAL}S").sum().reset_index()
#             average_table['time_index'] = average_table[tl][detect]['count']['index']
            average_table[tl][detect]['counts'].drop('index', axis=1, inplace=True)
            average_table[tl][detect]['counts'] = average_table[tl][detect]['counts'] * HISTORICAL_AGG_INTERVAL / AGG_INTERVAL

average_table['time_index'] = pd.DataFrame(index=time_index).resample(f"{AGG_INTERVAL}S").interpolate().reset_index()['index']
start_time = pd.to_datetime(START_TIME)
average_table['time_index'] = average_table['time_index'].apply(lambda x: x.replace(day=start_time.day, month=start_time.month, year=start_time.year))

63082002
63082003
63082004


## Plot the Dector Values

In [37]:
def calibration_plot(tl, detector, show_image=True, show_legend=True, secondary=False, hourly=True, write_image=True):
    
    
    
    
    day_values = rw_df[f"{tl}_{detector}"]
    mean_std, mean_index = average_table[tl][detector]['counts'], average_table['time_index']
    sumo_detect_volume = sumo_df[f"{tl}_{detector}"]

    sumo_detect_volume = sumo_detect_volume
    colors = ['#003f5c', '#bc5090', '#ff6361', '#ffa600', '#58508d'] 

    fig = go.Figure()
    
    # get the hourly multiplier
    hourly_multiplier = 3600 / AGG_INTERVAL if hourly else 1
    

    one_sigma = [
                 (day_values.values  * hourly_multiplier) + mean_std['std'],
                 (day_values.values  * hourly_multiplier) - mean_std['std']
                 ]

    nine_five_conf = [
                 (day_values.values  * hourly_multiplier) + 1.96 * mean_std['std'],
                 (day_values.values  * hourly_multiplier) - 1.96 * mean_std['std']
                 ]

    

    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=mean_index,
            y=nine_five_conf[0].values,
            name="Criteria I.",
            line=dict(color=colors[3], width=3)
        ),

    )

    fig.add_trace(
        go.Scatter(
            x=mean_index,
            y=nine_five_conf[1].values,
            name="Criteria I",
            showlegend=False,
            line=dict(color=colors[3], width=3),
            fill='tonexty'
        ),
    )


    fig.add_trace(
        go.Scatter(
            x=mean_index,
            y=one_sigma[0].values,
            name="Criteria II.",
            line=dict(color=colors[1], width=3)
        ),

    )

    fig.add_trace(
        go.Scatter(
            x=mean_index,
            y=one_sigma[1].values,
            name="Criteria II",
            showlegend=False,
            line=dict(color=colors[1], width=3),
            fill='tonexty'
        ),
    )

    fig.add_trace(
            go.Scatter(
                x=mean_index,
                y=day_values.values * hourly_multiplier,
                line=dict(width=5, color=colors[0]),
                name="Representative Day",
            ),
    )

    fig.add_trace(
            go.Scatter(
                x=sumo_detect_volume.index,
                y=sumo_detect_volume.values * hourly_multiplier,
                name="Simulation",
                mode='markers',
                marker=dict(
                            color='red',
                            symbol='diamond',
                            size=15,
                            line=dict(
                                color='black',
                                width=2
                            ),
                )
            ),
        )

    fig.update_yaxes(title_text="Volume [Vehicles/Hour]")

    fig.update_layout(
        template='simple_white',
        font_family='helvetica',
        font_size=26,
        height=600,
        width=1200,
        legend=dict(
            yanchor="top", y=1.0, xanchor="right", x=1, orientation='h'
        ),
        yaxis=dict(showgrid=True, range=[0, 250] if secondary else [0, 2000]),
        xaxis=dict(tickangle=15),
    )
    fig.update_layout(showlegend=show_legend)

    if show_image:
        fig.show()
    if write_image:
        (experiment_path / 'images' ).mkdir(parents=True, exist_ok=True)

        fig.write_image(Path(experiment_path) / "images" / f"{tl}-{detector}.png")

In [38]:
# TL = AIRPORT
# DETECTOR = 'WB'
calibration_plot(AIRPORT, 'WB')

### Printing All Calibration Plots

In [39]:
for pretty_tl, tl in zip(['TL1','TL2','TL3'], TRAFFIC_LIGHTS):
    i = 0
    for pretty_detect, detect in zip(['WB', 'EB', 'SB', 'NB'], ['WB', 'EB', 8, 4]):
        try:
            calibration_plot(tl, detect, show_image=True, show_legend=True, secondary=True if detect in [8, 4] else False )
            print(f"{{{tl}-{detect}.png/{pretty_tl}-{pretty_detect}}},%")
        except KeyError:
            pass
        i += 1 
# shutil.make_archive('images', 'zip', 'images')

{63082002-WB.png/TL1-WB},%


{63082002-EB.png/TL1-EB},%


{63082002-4.png/TL1-NB},%


{63082003-WB.png/TL2-WB},%


{63082003-EB.png/TL2-EB},%


...

{63082003-4.png/TL2-NB},%


{63082004-WB.png/TL3-WB},%


{63082004-EB.png/TL3-EB},%


{63082004-8.png/TL3-SB},%


{63082004-4.png/TL3-NB},%


## USDOT Calibration

In [40]:
min_day = '2020-02-24'

### Criterion I. Comparing to 1.96 sigma

In [41]:
in_out_196_sigma = {}
for tl_id in TRAFFIC_LIGHTS:
    in_out_196_sigma[tl_id] = {}
    for detector_id, detect_dict in average_table[tl_id].items():
        day_counts = rw_df[f"{tl_id}_{detector_id}"].values * 3600 / AGG_INTERVAL
        std_counts = detect_dict['counts']['std'].values
        one_sigma = [
             day_counts + 1.96 * std_counts,
             day_counts - 1.96 * std_counts
             ]
        inside = ((sumo_df[f"{tl_id}_{detector_id}"] * 3600 / AGG_INTERVAL) > one_sigma[1]) & ((sumo_df[f"{tl_id}_{detector_id}"] * 3600 / AGG_INTERVAL) < one_sigma[0])
        inside = inside * 1
        in_out_196_sigma[tl_id][detector_id] = sum(inside) / len(inside)
        print(f"Traffic light {tl_id} and {detector_id} are {in_out_196_sigma[tl_id][detector_id] * 100}% inside 1.96 sigma")

Traffic light 63082002 and 4 are 94.73684210526315% inside 1.96 sigma
Traffic light 63082002 and 5 are 97.36842105263158% inside 1.96 sigma
Traffic light 63082002 and 9 are 97.36842105263158% inside 1.96 sigma
Traffic light 63082002 and 10 are 100.0% inside 1.96 sigma
Traffic light 63082002 and 11 are 94.73684210526315% inside 1.96 sigma
Traffic light 63082002 and 12 are 94.73684210526315% inside 1.96 sigma
Traffic light 63082002 and EB are 100.0% inside 1.96 sigma
Traffic light 63082002 and WB are 97.36842105263158% inside 1.96 sigma
Traffic light 63082003 and 1 are 92.10526315789474% inside 1.96 sigma
Traffic light 63082003 and 3 are 89.47368421052632% inside 1.96 sigma
Traffic light 63082003 and 4 are 39.473684210526315% inside 1.96 sigma
Traffic light 63082003 and 5 are 86.8421052631579% inside 1.96 sigma
Traffic light 63082003 and 7 are 97.36842105263158% inside 1.96 sigma
Traffic light 63082003 and 8 are 92.10526315789474% inside 1.96 sigma
Traffic light 63082003 and 9 are 97.368

### Criterion II. Comparing to 1 sigma



In [42]:
in_out_1_sigma = {}
for tl_id in TRAFFIC_LIGHTS:
    in_out_1_sigma[tl_id] = {}
    for detector_id, detect_dict in average_table[tl_id].items():
        day_counts = rw_df[f"{tl_id}_{detector_id}"].values * 3600 / AGG_INTERVAL
        std_counts = detect_dict['counts']['std'].values
        one_sigma = [
             day_counts + std_counts,
             day_counts - std_counts
             ]
        inside = ((sumo_df[f"{tl_id}_{detector_id}"] * 3600 / AGG_INTERVAL) > one_sigma[1]) & ((sumo_df[f"{tl_id}_{detector_id}"] * 3600 / AGG_INTERVAL) < one_sigma[0])
        inside = inside * 1
        in_out_1_sigma[tl_id][detector_id] = sum(inside) / len(inside)
        print(f"Traffic light {tl_id} and {detector_id} are {in_out_1_sigma[tl_id][detector_id] * 100}% inside 1 sigma")

Traffic light 63082002 and 4 are 55.26315789473685% inside 1 sigma
Traffic light 63082002 and 5 are 65.78947368421053% inside 1 sigma
Traffic light 63082002 and 9 are 65.78947368421053% inside 1 sigma
Traffic light 63082002 and 10 are 78.94736842105263% inside 1 sigma
Traffic light 63082002 and 11 are 81.57894736842105% inside 1 sigma
Traffic light 63082002 and 12 are 63.1578947368421% inside 1 sigma
Traffic light 63082002 and EB are 86.8421052631579% inside 1 sigma
Traffic light 63082002 and WB are 68.42105263157895% inside 1 sigma
Traffic light 63082003 and 1 are 47.368421052631575% inside 1 sigma
Traffic light 63082003 and 3 are 52.63157894736842% inside 1 sigma
Traffic light 63082003 and 4 are 21.052631578947366% inside 1 sigma
Traffic light 63082003 and 5 are 55.26315789473685% inside 1 sigma
Traffic light 63082003 and 7 are 52.63157894736842% inside 1 sigma
Traffic light 63082003 and 8 are 42.10526315789473% inside 1 sigma
Traffic light 63082003 and 9 are 71.05263157894737% insid

### Criterion III: Bounded Dynamic Absolute Error (BDAE)

In [43]:
in_out_bounded = {}
for tl_id in TRAFFIC_LIGHTS:
    in_out_bounded[tl_id] = {}
    for detector_id, detect_dict in average_table[tl_id].items():
        BDAE = detect_dict['BDAE']
        N_T = len(detect_dict['counts'][min_day])
        sumo_counts = (sumo_df[f"{tl_id}_{detector_id}"] * 3600 / AGG_INTERVAL).values
        rw_counts = (rw_df[f"{tl_id}_{detector_id}"] * 3600 / AGG_INTERVAL).values
        print(BDAE, sum(abs(sumo_counts - rw_counts)) / N_T)
        inside = BDAE >= sum(abs(sumo_counts - rw_counts)) / N_T
        in_out_bounded[tl_id][detector_id] = inside
        print(f"Traffic light {tl_id} and {detector_id} detector {'are' if in_out_bounded[tl_id][detector_id] else 'are not'} inside BDAE") 

130.59539170506912 10.421052631578947
Traffic light 63082002 and 4 detector are inside BDAE
206.35576036866357 23.68421052631579
Traffic light 63082002 and 5 detector are inside BDAE
561.9096774193547 64.57894736842105
Traffic light 63082002 and 9 detector are inside BDAE
549.3124423963134 49.26315789473684
Traffic light 63082002 and 10 detector are inside BDAE
533.7179723502304 45.0
Traffic light 63082002 and 11 detector are inside BDAE
566.4221198156681 51.0
Traffic light 63082002 and 12 detector are inside BDAE
969.3013824884794 64.57894736842105
Traffic light 63082002 and EB detector are inside BDAE
950.941935483871 68.84210526315789
Traffic light 63082002 and WB detector are inside BDAE
169.84700460829492 22.105263157894736
Traffic light 63082003 and 1 detector are inside BDAE
120.49769585253456 12.947368421052632
Traffic light 63082003 and 3 detector are inside BDAE
142.694930875576 46.578947368421055
Traffic light 63082003 and 4 detector are inside BDAE
137.96129032258068 19.263

### Criterion IV. Bounded Dynamic Systematic Error

In [44]:
in_out_bounded_systematic = {}
for tl_id in TRAFFIC_LIGHTS:
    in_out_bounded_systematic[tl_id] = {}
    for detector_id, detect_dict in average_table[tl_id].items():
        BDAE = detect_dict['BDAE']
        N_T = len(detect_dict['counts'][min_day])

        sumo_counts = (sumo_df[f"{tl_id}_{detector_id}"] * 3600 / AGG_INTERVAL).values
        rw_counts = (rw_df[f"{tl_id}_{detector_id}"] * 3600 / AGG_INTERVAL).values

        inside = (BDAE*1/3) >= abs(sum(sumo_counts - rw_counts) / N_T)
        in_out_bounded_systematic[tl_id][detector_id] = inside
        print(f"Traffic light {tl_id} and {detector_id} detector {'are' if in_out_bounded_systematic[tl_id][detector_id] else 'are not'} inside systematic BDAE") 

Traffic light 63082002 and 4 detector are inside systematic BDAE
Traffic light 63082002 and 5 detector are inside systematic BDAE
Traffic light 63082002 and 9 detector are inside systematic BDAE
Traffic light 63082002 and 10 detector are inside systematic BDAE
Traffic light 63082002 and 11 detector are inside systematic BDAE
Traffic light 63082002 and 12 detector are inside systematic BDAE
Traffic light 63082002 and EB detector are inside systematic BDAE
Traffic light 63082002 and WB detector are inside systematic BDAE
Traffic light 63082003 and 1 detector are inside systematic BDAE
Traffic light 63082003 and 3 detector are inside systematic BDAE
Traffic light 63082003 and 4 detector are inside systematic BDAE
Traffic light 63082003 and 5 detector are inside systematic BDAE
Traffic light 63082003 and 7 detector are inside systematic BDAE
Traffic light 63082003 and 8 detector are inside systematic BDAE
Traffic light 63082003 and 9 detector are inside systematic BDAE
Traffic light 630820

In [45]:
multi_index = [(tl, detect) for tl in TRAFFIC_LIGHTS for detect in average_table[tl].keys()]
cols = pd.MultiIndex.from_tuples(multi_index)
pd_df = pd.DataFrame(index=['1.96 Sigma', '1 Sigma','Bounded Dynamic Absolute Error (BDAE)','Bounded Dynamic Systematic Error'], columns=cols)

In [46]:
# display all columns
pd.set_option('display.max_columns', None)

for tl in TRAFFIC_LIGHTS: 
    for detect in average_table[tl].keys():
#         if detect in ['EB', 'WB']: 
        pd_df.loc['1 Sigma', (tl, detect)] = round(in_out_1_sigma[tl][detect] * 100, 2)
        pd_df.loc['1.96 Sigma', (tl, detect)] = round(in_out_196_sigma[tl][detect] * 100, 2)
        pd_df.loc['Bounded Dynamic Absolute Error (BDAE)', (tl, detect)] = in_out_bounded[tl][detect]
        pd_df.loc['Bounded Dynamic Systematic Error', (tl, detect)] = in_out_bounded_systematic[tl][detect]
pd_df

63082002                              \
                                             4      5      9     10     11   
1.96 Sigma                               94.74  97.37  97.37  100.0  94.74   
1 Sigma                                  55.26  65.79  65.79  78.95  81.58   
Bounded Dynamic Absolute Error (BDAE)     True   True   True   True   True   
Bounded Dynamic Systematic Error          True   True   True   True   True   

                                                           63082003         \
                                          12     EB     WB        1      3   
1.96 Sigma                             94.74  100.0  97.37    92.11  89.47   
1 Sigma                                63.16  86.84  68.42    47.37  52.63   
Bounded Dynamic Absolute Error (BDAE)   True   True   True     True   True   
Bounded Dynamic Systematic Error        True   True   True     True   True   

                                                                          \
                                           4      5      7      8      9   
1.96 Sigma                             39.47  86.84  97.37  92.11  97.37   
1 Sigma                                21.05  55.26  52.63  42.11  71.05   
Bounded Dynamic Absolute Error (BDAE)   True   True   True   True   True   
Bounded Dynamic Systematic Error        True   True   True   True   True   

                                                                          \
                                          10     11     12     EB     WB   
1.96 Sigma                             100.0  100.0  97.37  100.0  100.0   
1 Sigma                                68.42  68.42  71.05  94.74  81.58   
Bounded Dynamic Absolute Error (BDAE)   True   True   True   True   True   
Bounded Dynamic Systematic Error        True   True   True   True   True   

                                      63082004                              \
                                             1      3      4      5      7   
1.96 Sigma                               89.47  100.0  86.84  94.74  76.32   
1 Sigma                                  52.63  76.32  71.05  73.68  15.79   
Bounded Dynamic Absolute Error (BDAE)     True   True   True   True   True   
Bounded Dynamic Systematic Error          True   True   True   True   True   

                                                                          \
                                           8      9     10     11     12   
1.96 Sigma                             97.37    0.0  100.0  97.37  92.11   
1 Sigma                                76.32    0.0  100.0  73.68  65.79   
Bounded Dynamic Absolute Error (BDAE)   True  False   True   True   True   
Bounded Dynamic Systematic Error        True  False   True   True   True   

                                                     
                                          EB     WB  
1.96 Sigma                             100.0  100.0  
1 Sigma                                94.74  97.37  
Bounded Dynamic Absolute Error (BDAE)   True   True  
Bounded Dynamic Systematic Error        True   True

In [24]:
# from sumolib.xml import parse_fast_structured

# iters = parse_fast_structured(
#     str(Path(s.OUTPUT_ROOT) / "fcd.final.out.xml"),
#     'timestep', ['time'],
#     {'vehicle': ['id']}
# )
# df = pd.DataFrame(iters)
# # df['vehicle'] = df['vehicle'].apply(lambda x: len(x))
# df = df.loc[df['time'].astype(float) % 5 <= 0].copy()
# df['vehicle'] = df['vehicle'].apply(lambda x: len(x))

In [25]:
# import plotly.express as px



# # scatter plot with lines and markers
# px.line(
#     df,
#     x="time",
#     y="vehicle",
#     title="Number of Vehicles in Simulation",
#     labels={"vehicle": "Number of Vehicles", "time": "Time"},
#     template="plotly_white",
#     width=800,
#     height=400,
# )